<a href="https://colab.research.google.com/github/Lucas-Jimenez-uribe/telecom_x/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

In [92]:
import pandas as pd
import numpy as np


In [93]:
df = pd.read_json('/content/TelecomX_Data.json')
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [94]:
df.shape

(7267, 6)

#🔧 Transformación

In [123]:
df.head()

,cliente_id,cliente_cancelado,cliente_genero,cliente_es_senior,cliente_tiene_pareja,cliente_tiene_dependientes,cliente_antiguedad_meses,servicio_telefonico,lineas_telefonicas_multiples,internet_tipo,...,internet_proteccion_dispositivo,internet_soporte_tecnico,internet_streaming_tv,internet_streaming_peliculas,cuenta_contrato,cuenta_facturacion_electronica,cuenta_metodo_pago,cuenta_cargos_mensuales,cuenta_cargos_totales,cuentas_diarias
0,0002-orfbo,0.0,female,0,1,1,9,1,0.0,dsl,...,0.0,1.0,1.0,0.0,one year,1,mailed check,65.6,593.30,2.19
1,0003-mknfe,0.0,male,0,0,0,9,1,1.0,dsl,...,0.0,0.0,0.0,1.0,month-to-month,0,mailed check,59.9,542.40,2.00
2,0004-tlhlj,1.0,male,0,0,0,4,1,0.0,fiber optic,...,1.0,0.0,0.0,0.0,month-to-month,1,electronic check,73.9,280.85,2.46
3,0011-igkff,1.0,male,1,1,0,13,1,0.0,fiber optic,...,1.0,0.0,1.0,1.0,month-to-month,1,electronic check,98.0,1237.85,3.27
4,0013-exchz,1.0,female,1,1,0,3,1,0.0,fiber optic,...,0.0,1.0,1.0,0.0,month-to-month,1,mailed check,83.9,267.40,2.80


In [ ]:
df.info()

In [ ]:
df_original = df.copy()

col_inicio = df.columns.get_loc('customer')

# Seleccionar todas las columnas desde esa en adelante
cols_a_normalizar = df.columns[col_inicio:]

# Extraer esas columnas como diccionarios por fila
registros = df[cols_a_normalizar].to_dict(orient='records')

# Normalizar
df = df_normalizado = pd.json_normalize(registros)

# Guardar las columnas que quedaron afuera
columnas_anteriores = df_original[['customerID', 'Churn']].reset_index(drop=True)

# Unirlas al nuevo dataframe normalizado
df = pd.concat([columnas_anteriores, df_normalizado], axis=1)

In [98]:
df.duplicated().sum()
df = df.drop_duplicates()

In [ ]:
df.isnull().sum()

In [121]:
# Lista de columnas donde vas a reemplazar NaN por 0
columnas_binarias = [
    'cliente_cancelado',
    'lineas_telefonicas_multiples',
    'internet_proteccion_dispositivo',
    'internet_soporte_tecnico',
    'internet_streaming_tv',
    'internet_streaming_peliculas',
]

# Reemplazar NaN por 0
df[columnas_binarias] = df[columnas_binarias].fillna(0)


In [101]:
df['account.Charges.Total'].unique()

df['account.Charges.Total'] = df['account.Charges.Total'].replace('', np.nan)

df['account.Charges.Total'] = df['account.Charges.Total'].str.strip()

df['account.Charges.Total'] = df['account.Charges.Total'].astype(float)



In [102]:
# Aplicar str.lower() y str.strip() a todas las columnas de texto
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.lower().str.strip()


In [ ]:
# Ver cuántas veces aparecen en cada columna
valores_nulos_falsos = ['n/a', 'na', 'null', 'none', '']

for valor in valores_nulos_falsos:
    print(f"--- Buscando '{valor}' ---")
    print((df == valor).sum())

In [ ]:
df.info()

In [ ]:
df.columns

In [106]:
df = df.rename(columns={
    'customerID': 'cliente_id',
    'Churn': 'cliente_cancelado',
    'customer.gender': 'cliente_genero',
    'customer.SeniorCitizen': 'cliente_es_senior',
    'customer.Partner': 'cliente_tiene_pareja',
    'customer.Dependents': 'cliente_tiene_dependientes',
    'customer.tenure': 'cliente_antiguedad_meses',

    'phone.PhoneService': 'servicio_telefonico',
    'phone.MultipleLines': 'lineas_telefonicas_multiples',

    'internet.InternetService': 'internet_tipo',
    'internet.OnlineSecurity': 'internet_seguridad_en_linea',
    'internet.OnlineBackup': 'internet_respaldo_en_linea',
    'internet.DeviceProtection': 'internet_proteccion_dispositivo',
    'internet.TechSupport': 'internet_soporte_tecnico',
    'internet.StreamingTV': 'internet_streaming_tv',
    'internet.StreamingMovies': 'internet_streaming_peliculas',

    'account.Contract': 'cuenta_contrato',
    'account.PaperlessBilling': 'cuenta_facturacion_electronica',
    'account.PaymentMethod': 'cuenta_metodo_pago',
    'account.Charges.Monthly': 'cuenta_cargos_mensuales',
    'account.Charges.Total': 'cuenta_cargos_totales'
})


In [ ]:
df.head()

In [108]:
df['cuentas_diarias'] = (df['cuenta_cargos_mensuales'] / 30).round(2)

In [109]:
columnas_binarias = ['cliente_cancelado', 'cliente_tiene_pareja', 'cliente_tiene_dependientes', 'servicio_telefonico', 'lineas_telefonicas_multiples','internet_proteccion_dispositivo', 'internet_soporte_tecnico','internet_streaming_tv','internet_streaming_peliculas','cuenta_facturacion_electronica' ]
for col in columnas_binarias:
    df[col] = df[col].map({'yes': 1, 'no': 0})



In [ ]:
for col in columnas_binarias:
    print(f"{col}: {df[col].unique()}")


#📊 Carga y análisis

#📄Informe final